<a href="https://colab.research.google.com/github/v281vy1/FUTURE_DS_01/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Reading and Cleaning**

In [1]:
# -------------------------
# 1. Import Libraries
# -------------------------
import pandas as pd
import numpy as np

In [6]:
# -------------------------
# 2. Load Dataset
# -------------------------
df = pd.read_csv('ecommerce_data.csv',encoding="ISO-8859-1")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12-01-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12-01-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12-01-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-01-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12-01-2010 08:26,3.39,17850.0,United Kingdom


In [7]:
# -------------------------
# 3. Check Data Info
# -------------------------
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261330 entries, 0 to 261329
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    261330 non-null  object 
 1   StockCode    261330 non-null  object 
 2   Description  260373 non-null  object 
 3   Quantity     261330 non-null  int64  
 4   InvoiceDate  261330 non-null  object 
 5   UnitPrice    261329 non-null  float64
 6   CustomerID   186186 non-null  float64
 7   Country      261329 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 16.0+ MB
None
            Quantity      UnitPrice     CustomerID
count  261330.000000  261329.000000  186186.000000
mean        9.271741       5.010345   15291.904934
std       214.008454     115.066400    1730.881715
min    -74215.000000       0.000000   12346.000000
25%         1.000000       1.250000   13854.000000
50%         3.000000       2.100000   15165.000000
75%        10.000000       4.150000

In [8]:
# -------------------------
# 4. Remove Duplicates
# -------------------------
df = df.drop_duplicates()
print("After removing duplicates:", df.shape)

After removing duplicates: (259230, 8)


In [9]:
# -------------------------
# 5. Handle Missing Values
# -------------------------
# Show % of missing values
print("Missing values per column:\n", df.isnull().mean() * 100)

# Handle Description (drop missing descriptions)
df = df.dropna(subset=['Description'])

# Handle UnitPrice (only 1 missing - drop it)
df = df.dropna(subset=['UnitPrice'])

# Handle CustomerID
# Option A: drop rows without CustomerID (if you want customer-level analysis)
df = df.dropna(subset=['CustomerID'])

# Option B: keep them (if focusing only on overall sales)
# df['CustomerID'] = df['CustomerID'].fillna("Unknown")

print("After handling missing values:", df.shape)

Missing values per column:
 InvoiceNo       0.000000
StockCode       0.000000
Description     0.369170
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000386
CustomerID     28.976585
Country         0.000386
dtype: float64
After handling missing values: (184114, 8)


In [10]:
# -------------------------
# 6. Convert Data Types
# -------------------------
# 1. Ensure OrderDate is datetime
if "InvoiceDate" in df.columns:
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# 2. Ensure numerical columns are correct type
if "Quantity" in df.columns:
    df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
if "UnitPrice" in df.columns:
    df["UnitPrice"] = pd.to_numeric(df["UnitPrice"], errors="coerce")

# 3. Convert CustomerID
if "CustomerID" in df.columns:
    # convert to string to avoid issues with float values like 17850.0
    df['CustomerID'] = df['CustomerID'].astype(str).str.replace('.0', '', regex=False)

In [11]:
# -------------------------
# 7. Remove Negative or Invalid Values
# -------------------------
# Drop negative or zero Quantity (returns/cancellations)
# Drop zero or negative prices
# Drop rows with Quantity <= 0 or Price <= 0
if "Quantity" in df.columns and "UnitPrice" in df.columns:
    df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]

print("After removing invalid values:", df.shape)

After removing invalid values: (179640, 8)


In [12]:
# -------------------------
# 8. Feature Engineering
# -------------------------

# Add Revenue column
df['Revenue'] = df['Quantity'] * df['UnitPrice']

# Extract date parts for trend analysis
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')

In [14]:
# -------------------------
# 9. Save Cleaned Data
# -------------------------
df.to_csv("cleaned_ecommerce.csv", index=False)
print(" Cleaned dataset saved as 'cleaned_ecommerce.csv'")

 Cleaned dataset saved as 'cleaned_ecommerce.csv'
